In [ ]:
using Plots
using Statistics
using FFTW
FFTW.set_provider!("mkl")
#import Pkg; Pkg.add("FileIO")
using FileIO
using Base.Threads
using Distributions
using LsqFit
#Pkg; Pkg.add("DistributedArrays")
println("nthreads = $(nthreads())")

In [ ]:
data_2 = load("fine_landau_undamped3522_200_alp2_n2_Th3th1_results.jld2")

In [ ]:
t_i, t_f, M, M_g, dt = data_2["par_evolv"]

In [ ]:

p0 = [7.90*10^(-5); 0.561111]
@. model(x, p) = p[1]*cos(x*p[2])^2 #* exp(-x*p[3]) #*cos(x*p[5] + p[6])

In [ ]:
t_series = [dt*(i-1)*((M-1)÷(M_g-1)) for i in 1:M_g];
fit = curve_fit(model, t_series, data_2["Energy_E"], p0);
fit.param

In [ ]:
p1 = 7.90*10^(-5); p2 = 0.561111
plot(t_series, data_2["Energy_E"])
#plot!(t_series, model(t_series, fit.param))
plot!(t_series, p1*cos.(p2*t_series).^2)

Fit for undamped with parameters:



$p1 = 7.90*10^{-5}$  $p2 = 0.561111$ for $E_E(t) = p1\; cos(p2*t)^2$


Los parámetros de fiteo son: 

### Caso undamped n = 4

    Lx = 39.738, Nx = 3522, Np = 200, Lp = 0.5

    α = 0.01
    n = 4 (chekeado que la longitud de onda es Lx/4)
    k = 2*π*n/Lx
    θ = 0.001

$E_K = a + b*cos(\omega*t + \alpha)^2$ 

Tenemos $a = 1.0005005$, $b = 0.0000196$ $\omega = 0.562$, $\alpha = -\pi/2$

$E_K = a' + d'*cos(\omega' * t + \alpha')$ 

Tenemos $a'= 1.0005103$, $b'= 0.0000196/2$ $\omega' = 0.562*2$ $\alpha'= - π$

$E_E = p_1 * cos(p_2 * t)^2

Tenemos $p_1 = 1.969*10^{-5} $, $p_2 = 0.5619$ 

### Caso undamped n = 2

Lx = 39.738, Nx = 3522, Np = 200, Lp = 0.5

    α = 0.01
    n = 2 (chekeado que la longitud de onda es Lx/2)
    k = 2*π*n/Lx
    θ = 0.001

Tenemos $p1 = 7.90*10^{-5}$  $p2 = 0.561111$ for $E_E(t) = p1\; cos(p2*t)^2$

$\omega = \; \sqrt{4π} * 0.561111 \;= \;1.989 $


**No entiendo porque no cambian las frequencias entre n = 2 y 4. Bueno, según SHARP comienza cuadrático, puede ser que este bien**

### Caso damped

    Lx = 7.455, Nx = 930, Np = 200, Lp = 0.5

    α = 0.01
    n = 15
    k = 2*π*n/Lx
    θ = 0.001

$E_E = b*cos(\omega*t)^2 e^{-\gamma * t}$ 

Tenemos: $b = 1.450 \; 10^{-7}$, $\omega = 1.512$, $\gamma = 0.03$


#### Note: 

Since we are using a $4\pi$ in the equation for E dot, we need to change E and t to get to the equations in the SHARP paper, for that we have to change time by a factor $\sqrt{4\pi}$. 

$$
\hat t = a t \;\;\;\; \partial_{\hat t} = \frac{1}{a} \partial_t 
$$

\begin{align*}
\partial_t f &= a \partial_{\hat t} f = E \partial_p f = a \hat{E} \partial_p f \\
\partial_t E &= a^2 \partial_{\hat t} \hat{E} = 4\pi S
\end{align*}

Therefore $a^2 = 4\pi$ (the hat variables are the ones in the SHARP paper), or $\hat t = \sqrt{4\pi} t$, or 
$\hat{\omega} = \omega /\sqrt{4\pi}$


Using this factor we get:

#### undamped: #### 

$\hat{\omega} = \;  0.562  / \sqrt{4π} \;= \;0.15828 $

#### damped: #### 

$\hat{\omega} = 1.512 / \sqrt{4π} = 0.42652$, $\hat{\gamma} =  0.03 / \sqrt{4π} = 0.008$


We also have: 

$$
\frac{\omega_d}{\omega_u} = 2.6905 \;\;\;\;\;\;\; \frac{\gamma}{\omega_u} = 0.053
$$

In [ ]:
 0.03 / sqrt(4π)
 0.03 / 0.562
 2.69 / 2

In [ ]:
plot(data_2["S_F"])

In [ ]:
data_4 = load("fine_landau_3522_200_alp2_n_Th3th1_results.jld2")

In [ ]:
plot(data_4["S_F"])

In [ ]:
p_4_1 = 1.969*10^(-5) ; p_4_2 = 0.5619
plot(t_series, data_4["Energy_E"])
plot!(t_series, p_4_1 * cos.(p_4_2 * t_series).^2)